In [1]:
import set_path

import numpy as np
import pandas as pd
import logging
import time
import glob

import src.utils.ahn_utils as ahn_utils
import src.utils.bgt_utils as bgt_utils
import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
import src.utils.plot_utils as plot_utils
import src.analysis.analysis_tools as analysis_tools

from armatuur_fuser import ArmatuurFuser

log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

logger = logging.getLogger(__name__)

**Load Data**

In [ ]:
# Load data.
tilecode = '2387_9699'

# Labelled LAS file (see Notebook 0).
las_file = '../datasets/Lairessestraat/pointcloud/labelled_' + tilecode + '.laz'
processed_las_file = '../datasets/Lairessestraat/pointcloud/processed_' + tilecode + '.laz'

In [ ]:
# Load pointcloud
pointcloud = las_utils.read_las(las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# colors
pc_colors =  np.vstack((pointcloud.red, pointcloud.blue, pointcloud.green)).T

# Mask
mask = np.ones(len(points), dtype=bool)

In [ ]:
# Load pointcloud
pointcloud = las_utils.read_las(processed_las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# colors
pc_colors =  np.vstack((pointcloud.red, pointcloud.blue, pointcloud.green)).T

# Mask
mask = np.ones(len(points), dtype=bool)

**Armatuur Extraction**

In [ ]:
armatuur_params={'width': (.25, 1.25), 'height': (.25, .6), 'z_offset':(0,.55), 'axis_offset': 0.1}

armatuur_extractor = ArmatuurFuser(15, armatuur_params=armatuur_params, min_cable_bending=2)

In [ ]:
label_mask = armatuur_extractor.get_label_mask(points, None, label_mask, tilecode)

**Plot Results**

In [ ]:
plot_utils.plot_pointcloud_o3d(points[label_mask], np.zeros(label_mask.sum()))

#### Batch Folder + Stats

In [25]:
def process_file(in_file, armatuur_params, folder_path, save_output):
        
    tilecode = las_utils.get_tilecode_from_filename(in_file)
    logger.info(f'Processing file {in_file}.')
    start = time.time()
    
    pointcloud = las_utils.read_las(in_file)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    true_labels = np.zeros((len(points),), dtype='uint16')
    true_labels[pointcloud.label > 14] = 15
    true_labels[pointcloud.label == 12] = 12
    true_mask = true_labels == 15

    pointcloud_labelled = las_utils.read_las(folder_path+'/processed/M3_'+tilecode+'.laz')
    labels_pred = pointcloud_labelled.label
    mask = (labels_pred == 11)
    mask = (true_labels == 12)

    armatuur_extractor = ArmatuurFuser(15, armatuur_params=armatuur_params, min_cable_bending=2)
    label_mask = armatuur_extractor.get_label_mask(points, None, mask, tilecode)
    labels_pred[label_mask] = 15

    duration = time.time() - start
    stats, report = analysis_tools.get_cable_stats_m4(labels_pred, true_mask)
    report['time'] = duration
    report['tilecode'] = tilecode

    logger.info('STATISTICS\n' + stats)
    logger.info(f'File processed in {duration:.2f}s.')

    if save_output:
        outpath = folder_path + '/processed/M4_' + tilecode + '.laz'
        las_utils.label_and_save_las(pointcloud, labels_pred, outpath)

    return report

    

In [26]:
def process_folder(folder_path, save_output=False):

    # Parameters
    armatuur_params = {
        'width': (.25, 1.25),
        'height': (.25, .6),
        'z_offset':(0,.55),
        'axis_offset': 0.1
    }

    results = []

    for file in glob.iglob(folder_path + '/pointcloud/labelled_*.laz'):
        report = process_file(file, armatuur_params, folder_path, save_output)
        results.append(report)

    return pd.DataFrame(results)



In [27]:
df = process_folder('../datasets/Lairessestraat', True)

INFO - 2022-04-26 17:22:12,788 - 2304772302 - Processing file ../datasets/Lairessestraat/pointcloud/labelled_2394_9702.laz.
INFO - 2022-04-26 17:22:16,104 - armatuur_fuser - Armatuur fuser (label=15)
INFO - 2022-04-26 17:22:16,647 - 2304772302 - STATISTICS
Labelled 514 / 517
INFO - 2022-04-26 17:22:16,648 - 2304772302 - File processed in 3.81s.
INFO - 2022-04-26 17:22:18,261 - 2304772302 - Processing file ../datasets/Lairessestraat/pointcloud/labelled_2388_9700.laz.
INFO - 2022-04-26 17:22:22,341 - armatuur_fuser - Armatuur fuser (label=15)
INFO - 2022-04-26 17:22:24,079 - 2304772302 - STATISTICS
Labelled 1749 / 1749
INFO - 2022-04-26 17:22:24,080 - 2304772302 - File processed in 5.76s.
INFO - 2022-04-26 17:22:26,206 - 2304772302 - Processing file ../datasets/Lairessestraat/pointcloud/labelled_2389_9700.laz.
INFO - 2022-04-26 17:22:30,615 - armatuur_fuser - Armatuur fuser (label=15)
INFO - 2022-04-26 17:22:31,399 - 2304772302 - STATISTICS
Labelled 427 / 427
INFO - 2022-04-26 17:22:31,3

In [28]:
df

,total_pts,armatuur_pts,recall_m3,precision_m3,f1_score_m3,time,tilecode
0,4248444,517,0.994197,1.0,0.997090,3.809476,2394_9702
1,5769704,1749,1.000000,1.0,1.000000,5.755863,2388_9700
2,4853053,427,1.000000,1.0,1.000000,5.135470,2389_9700
3,2194654,259,0.996139,1.0,0.998066,2.418773,2391_9701
4,2788759,877,0.828962,1.0,0.906484,3.217150,2386_9699
5,4379344,1597,1.000000,1.0,1.000000,5.157902,2387_9699
6,6508713,462,1.000000,1.0,1.000000,7.595182,2387_9700


In [29]:
labels = las_utils.read_las('../datasets/Lairessestraat/processed/M1_2386_9699.laz').label

pointcloud = las_utils.read_las('../datasets/Lairessestraat/processed/M4_2386_9699.laz')
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels[pointcloud.label==11]=11
labels[pointcloud.label==13]=13
labels[pointcloud.label==15]=15

plot_utils.plot_pointcloud_o3d(points, labels, exclude_labels=(1,2))